# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [58]:
# import libraries
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import re

import nltk
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split

from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer


from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report

In [30]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\stuar\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\stuar\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\stuar\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\wordnet.zip.


True

In [2]:
db_path=r'data/DisasterResponse.db'

In [6]:
# load data from database
engine = create_engine(f'sqlite:///{db_path}')

In [34]:
df = pd.read_sql("SELECT * FROM DisasterResponse",engine)
df.index=df['id']
X = df[['message','original','genre']]
y = df.drop(columns=['id','message','original','genre'])

In [35]:
X

,message,original,genre
id,,,
2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct
7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct
8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct
9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct
12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct
...,...,...,...
30261,The training demonstrated how to enhance micro...,None,news
30262,A suitable candidate has been selected and OCH...,None,news
30263,"Proshika, operating in Cox's Bazar municipalit...",None,news


In [36]:
y

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
id,,,,,,,,,,,,,,,,,,,,,
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
8,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30261,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
30262,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
30263,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 2. Write a tokenization function to process your text data

In [18]:
def tokenize(text):
    #lower case and add remove punctuation
    text=re.sub(r"[^a-zA-Z0-9]"," ",text.lower())
    
    #tokenize text
    tokens=word_tokenize(text)
    
    #remove stop words and lemmatize
    tokens = [WordNetLemmatizer().lemmatize(w)  for w in tokens if w not in stopwords.words("english")]
    
    return tokens
        

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [45]:
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier()))
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [49]:
#split into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [53]:
y_train.shape,X_train.shape

((19662, 36), (19662, 3))

In [55]:
pipeline.fit(X_train['message'],y_train)

Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x0000026A20F45790>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier()))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [65]:
y_pred=pipeline.predict(X_test['message'])

In [71]:
len(y_pred[:,1])

6554

In [83]:
y_test.iloc[:,1].name

'request'

In [87]:
for i in range(y_pred.shape[1]):
    
    y_true_col=y_test.iloc[:,i]
    y_pred_col=y_pred[:,i]
    
    category=y_true_col.name
    print("#"*60)
    print(f"Classification report for category: {category}\n")
    print(classification_report(y_true_col, y_pred_col))

############################################################
Classification report for category: related

              precision    recall  f1-score   support

           0       0.72      0.41      0.52      1554
           1       0.83      0.95      0.89      4956
           2       0.49      0.41      0.44        44

    accuracy                           0.82      6554
   macro avg       0.68      0.59      0.62      6554
weighted avg       0.81      0.82      0.80      6554

############################################################
Classification report for category: request

              precision    recall  f1-score   support

           0       0.91      0.98      0.94      5459
           1       0.82      0.52      0.64      1095

    accuracy                           0.90      6554
   macro avg       0.86      0.75      0.79      6554
weighted avg       0.90      0.90      0.89      6554

############################################################
Classification repo

C:\Users\stuar\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\stuar\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\stuar\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.97      1.00      0.98      6335
           1       1.00      0.02      0.04       219

    accuracy                           0.97      6554
   macro avg       0.98      0.51      0.51      6554
weighted avg       0.97      0.97      0.95      6554

############################################################
Classification report for category: death

              precision    recall  f1-score   support

           0       0.96      1.00      0.98      6281
           1       0.76      0.14      0.24       273

    accuracy                           0.96      6554
   macro avg       0.86      0.57      0.61      6554
weighted avg       0.96      0.96      0.95      6554

############################################################
Classification report for category: other_aid

              precision    recall  f1-score   support

           0       0.88      1.00      0.93      5722
           1       0.55  

C:\Users\stuar\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\stuar\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\stuar\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\stuar\anaconda3\lib\site-packages\sklearn\metr

### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
parameters = 

cv = 

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.